# 2. Sistema de Recomendación (Market Basket Analysis)

### 2.1. Selección del Dataset
Originalmente se propuso utilizar el dataset *Amazon Reviews*. Sin embargo, debido a las recientes actualizaciones de seguridad en la librería `datasets` de Hugging Face (error `trust_remote_code`), el script de carga de dicho dataset ha quedado obsoleto y bloqueado.

Para cumplir con el objetivo técnico de desarrollar un **Sistema de Recomendación basado en Reglas de Asociación**, se migró al dataset estándar de la industria: **"Online Retail" (UCI Machine Learning Repository)**.

Este dataset contiene transacciones reales de una tienda minorista, lo que permite aplicar el algoritmo **Apriori** para detectar patrones de compra cruzada (*Cross-selling*).

### 2.2. Metodología
* **Algoritmo:** Apriori.
* **Filtro:** Se analizaron las transacciones de **Francia** para optimizar el rendimiento computacional.
* **Métrica Clave (Lift):** Indica la fuerza de la asociación. Un Lift > 1 implica que la compra de A estimula la compra de B.

In [3]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import warnings

warnings.filterwarnings('ignore')

print("⬇️  Cargando dataset 'Online Retail' desde UCI Archive...")
# URL directa al archivo Excel
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
df = pd.read_excel(url, engine='openpyxl')

# --- LIMPIEZA DE DATOS ---
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')] # Quitar devoluciones

# Filtrar por país (Francia) para la matriz
basket = (df[df['Country'] == "France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# Codificación One-Hot
def encode_units(x):
    if x <= 0: return 0
    if x >= 1: return 1

basket_sets = basket.applymap(encode_units)
if 'POSTAGE' in basket_sets.columns:
    basket_sets.drop('POSTAGE', inplace=True, axis=1)

print(f"✅ Matriz generada: {basket_sets.shape[0]} transacciones.")

# --- ALGORITMO APRIORI ---
# Soporte mínimo 7%
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

# --- REGLAS DE ASOCIACIÓN ---
# Lift mínimo de 1 (Asociación positiva)
reglas = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
reglas = reglas.sort_values(by='confidence', ascending=False)

# Mostrar las reglas más fuertes
print("\n🏆 Top Reglas Generadas:")
display(reglas[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head())

⬇️  Cargando dataset 'Online Retail' desde UCI Archive...
✅ Matriz generada: 392 transacciones.

🏆 Top Reglas Generadas:


,antecedents,consequents,support,confidence,lift
20,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.099490,0.975000,7.644000
22,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.099490,0.975000,7.077778
19,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.122449,0.960000,6.968889
18,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.122449,0.888889,6.968889
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.079082,0.837838,8.642959


### 2.3. Motor de Recomendación en Acción

A continuación, se implementa una función que toma un producto de entrada y sugiere complementos basados en el historial de ventas.

**Escenario de prueba:** Un cliente añade a su carrito **"PLASTERS IN TIN CIRCUS PARADE"** (Curitas con diseño de circo).

In [4]:
def recomendar_producto(item_input):
    print(f"🔎 Cliente comprando: '{item_input}'")
    
    # Buscar reglas donde item_input esté en los antecedentes
    recs = reglas[reglas['antecedents'].apply(lambda x: item_input in x)]
    
    if recs.empty:
        print("⚠️ No hay recomendaciones para este producto.")
        return
    
    # Ordenar por Lift
    top_recs = recs.sort_values(by='lift', ascending=False).head(3)
    
    print("🌟 EL SISTEMA SUGIERE:")
    for idx, row in top_recs.iterrows():
        consecuente = list(row['consequents'])[0]
        lift = round(row['lift'], 2)
        confianza = round(row['confidence'] * 100, 1)
        print(f"   -> {consecuente} (Lift: {lift} | Confianza: {confianza}%)")

# PRUEBA
producto_prueba = 'PLASTERS IN TIN CIRCUS PARADE'
if producto_prueba in basket_sets.columns:
    recomendar_producto(producto_prueba)

🔎 Cliente comprando: 'PLASTERS IN TIN CIRCUS PARADE'
🌟 EL SISTEMA SUGIERE:
   -> PLASTERS IN TIN SPACEBOY (Lift: 3.85 | Confianza: 53.0%)
   -> PLASTERS IN TIN WOODLAND ANIMALS (Lift: 3.55 | Confianza: 60.6%)


### 2.4. Conclusiones

Los resultados muestran una asociación lógica y fuerte:
* Al comprar **Curitas de Circo**, el sistema recomienda **Curitas de Espacio (Spaceboy)** y **Curitas de Animales (Woodland)** con un **Lift superior a 3.5**.
* Esto tiene sentido comercial: el cliente es probablemente un padre comprando artículos para niños, por lo que ofrecerle variaciones del mismo tipo de producto aumenta la probabilidad de venta.
* **Estrategia:** La empresa debería colocar estos productos visualmente juntos en la tienda online o crear un "Pack de Curitas Variadas".